<a href="https://colab.research.google.com/github/Simon-Pu/Temp/blob/master/Pytorch_onnx_TF_pb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
tf.__version__
!pip install onnx
!git clone https://github.com/onnx/onnx-tensorflow.git

import os
os.chdir('onnx-tensorflow')
!pip install -e .

import onnx
from onnx_tf.backend import prepare


     |████████████████████████████████| 7.4MB 6.3MB/s 
Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 4542 (delta 14), reused 6 (delta 0), pack-reused 4507
Receiving objects: 100% (4542/4542), 1.15 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (3413/3413), done.
Obtaining file:///content/onnx-tensorflow
  Running setup.py develop for onnx-tf


/content/onnx-tensorflow/onnx_tf/common/__init__.py:96: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)


In [7]:
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torchvision.models as models

resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
resnext50_32x4d.eval()
x = torch.randn(1, 3, 224, 224)
optional_args = dict(keep_initializers_as_inputs=True)
torch.onnx.export(resnext50_32x4d,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "resnext50_32x4d.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  verbose=False,
                  opset_version=7,          # the ONNX version to export the model to
                  # do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  **optional_args,
                  )
onnx_model = onnx.load("resnext50_32x4d.onnx")
onnx.checker.check_model(onnx_model)  # assuming throw on error
print("==> Passed")


==> Passed


In [14]:
import onnx
import warnings
from onnx_tf.backend import prepare

onnx_model = onnx.load("resnext50_32x4d.onnx")
tf_rep = prepare(onnx_model) # Import the ONNX model to Tensorflow
print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model
output_from_pytorch_model = resnext50_32x4d(x)
output_from_pytorch_model = output_from_pytorch_model.cpu().data.numpy()
#print('output_pytorch = {}'.format(output_from_pytorch_model))
output_onnx_tf = tf_rep.run(x)
#print('output_onnx_tf = {}'.format(output_onnx_tf))
mse = np.mean((output_onnx_tf - output_from_pytorch_model)**2)
print('MSE Error between ONNX-TF and Pytorh = {}'.format(mse))
tf_rep.export_graph("resnext50_32x4d_graph.pb")

2020-05-15 07:55:21,941 - onnx-tf - INFO - Fail to get since_version of Acosh in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,943 - onnx-tf - INFO - Fail to get since_version of Asinh in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,944 - onnx-tf - INFO - Fail to get since_version of Atanh in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,946 - onnx-tf - INFO - Fail to get since_version of BitShift in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,948 - onnx-tf - INFO - Fail to get since_version of Compress in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,950 - onnx-tf - INFO - Unknown op ConstantFill in domain `ai.onnx`.
2020-05-15 07:55:21,951 - onnx-tf - INFO - Fail to get since_version of ConstantOfShape in domain `` with max_inclusive_version=7. Set to 1.
2020-05-15 07:55:21,953 - onnx-tf - INFO - Fail to get since_version of ConvInteger in domain `` with max_incl

['input']
-----
['output']
-----
{'bn1.bias': <tf.Tensor 'bn1.bias:0' shape=(64,) dtype=float32>, 'bn1.num_batches_tracked': <tf.Tensor 'bn1.num_batches_tracked:0' shape=() dtype=int64>, 'bn1.running_mean': <tf.Tensor 'bn1.running_mean:0' shape=(64,) dtype=float32>, 'bn1.running_var': <tf.Tensor 'bn1.running_var:0' shape=(64,) dtype=float32>, 'bn1.weight': <tf.Tensor 'bn1.weight:0' shape=(64,) dtype=float32>, 'conv1.weight': <tf.Tensor 'conv1.weight:0' shape=(64, 3, 7, 7) dtype=float32>, 'fc.bias': <tf.Tensor 'fc.bias:0' shape=(1000,) dtype=float32>, 'fc.weight': <tf.Tensor 'fc.weight:0' shape=(1000, 2048) dtype=float32>, 'layer1.0.bn1.bias': <tf.Tensor 'layer1.0.bn1.bias:0' shape=(128,) dtype=float32>, 'layer1.0.bn1.num_batches_tracked': <tf.Tensor 'layer1.0.bn1.num_batches_tracked:0' shape=() dtype=int64>, 'layer1.0.bn1.running_mean': <tf.Tensor 'layer1.0.bn1.running_mean:0' shape=(128,) dtype=float32>, 'layer1.0.bn1.running_var': <tf.Tensor 'layer1.0.bn1.running_var:0' shape=(128,) 

In [22]:
# step 3, check if tf.pb is right.
with tf.Graph().as_default():
    graph_def = tf.compat.v1.GraphDef()
    with open("resnext50_32x4d_graph.pb", "rb") as f:
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name="")
        with tf.compat.v1.Session() as sess:
            #init = tf.initialize_all_variables()
            init =  tf.compat.v1.global_variables_initializer()
            #sess.run(init)
            
            # print all ops, check input/output tensor name.
            # uncomment it if you donnot know io tensor names.
            '''
            print('-------------ops---------------------')
            op = sess.graph.get_operations()
            for m in op:
                print(m.values())
            print('-------------ops done.---------------------')
            '''
            input_x = sess.graph.get_tensor_by_name("input:0") # input
            outputs1 = sess.graph.get_tensor_by_name('output:0') # 5
            output_tf_pb = sess.run([outputs1], feed_dict={input_x:x})
            #output_tf_pb = sess.run([outputs1, outputs2], feed_dict={input_x:np.random.randn(1, 3, 224, 224)})
            #print('output_tf_pb = {}'.format(output_tf_pb))

mse = np.mean((output_tf_pb - output_from_pytorch_model)**2)
print('MSE Error between tf_pb and Pytorh = {}'.format(mse))
#mse = np.mean((output_tf_pb - output_onnx_tf )**2)
#print('MSE Error between tf_pb and ONNX-TF = {}'.format(mse))                       

MSE Error between tf_pb and Pytorh = 1.4293081692165477e-12
